In [2]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time, os
import json

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=OPENAI_API_KEY)




In [6]:
INPUT_PATH = "reviews_openai_analysis_partial.csv"

df = pd.read_csv(INPUT_PATH)

In [7]:
df.head()

,category_type,product,search_term,title,text,date,score,comments,subreddit,url,Unnamed: 10,Unnamed: 11,summarization,openai_analysis
0,AI image generation models,DALL·E,best settings,Best tools for a beginner? (esp people/faces),Which tools do you think are best for a beginn...,2024-08-09 01:06:29,1,1,aiArt,https://reddit.com/r/aiArt/comments/1enk630/be...,NaN,NaN,Which tools do you think are best for a beginn...,"{\n ""sentiment"": ""Neutral"",\n ""key_points"": ..."
1,AI image generation models,DALL·E,vs DALL·E,AI Tools to Generate Images of People with Pro...,Hi everyone! 👋\n\nI’m looking for an AI tool t...,2025-01-13 23:31:01,0,4,StableDiffusion,https://reddit.com/r/StableDiffusion/comments/...,NaN,NaN,Hi everyone! Im looking for an AI tool that ca...,"{\n ""sentiment"": ""Neutral"",\n ""key_points"": ..."
2,AI image generation models,DALL·E,AI art workflow,Friday update for r/StableDiffusion - all the ...,* **SKYBOX AI**: create 360° worlds with one i...,2024-09-06 11:03:37,182,20,StableDiffusion,https://reddit.com/r/StableDiffusion/comments/...,NaN,NaN,GITHUB ComfyUI v0.2.0 Enhanced LivePortrait Up...,"{\n ""sentiment"": ""Neutral"",\n ""key_points"": ..."
3,AI image generation models,DALL·E,first impressions,Help with LoRA Training: Faint Stripes Appeari...,"Hi everyone!\n\nFirst off, thanks to this amaz...",2024-09-11 02:08:10,1,0,StableDiffusion,https://reddit.com/r/StableDiffusion/comments/...,NaN,NaN,"Hi everyone! First off, thanks to this amazing...","{\n ""sentiment"": ""Neutral"",\n ""key_points"": ..."
4,AI image generation models,DALL·E,first impressions,Two 80-something journalists tried ChatGPT. Th...,When two octogenarian buddies named Nick disco...,2024-07-11 23:28:13,61,102,ArtificialInteligence,https://reddit.com/r/ArtificialInteligence/com...,NaN,NaN,When two octogenarian buddies named Nick disco...,"{\n ""sentiment"": ""Bad"",\n ""key_points"": [""Tw..."


In [4]:
df.describe

<bound method NDFrame.describe of                    category_type           product        search_term  \
0     AI image generation models            DALL·E      best settings   
1     AI image generation models            DALL·E          vs DALL·E   
2     AI image generation models            DALL·E    AI art workflow   
3     AI image generation models            DALL·E  first impressions   
4     AI image generation models            DALL·E  first impressions   
...                          ...               ...                ...   
1495  AI image generation models  Stable Diffusion      best settings   
1496  AI image generation models  Stable Diffusion              using   
1497  AI image generation models  Stable Diffusion             tested   
1498  AI image generation models  Stable Diffusion           workflow   
1499  AI image generation models  Stable Diffusion           workflow   

                                                  title  \
0        Best tools for a begi

In [30]:
df['openai_analysis'].isnull().sum()

np.int64(493)

In [6]:
df[df['openai_analysis'].isnull()]

,category_type,product,search_term,title,text,date,score,comments,subreddit,url,Unnamed: 10,Unnamed: 11,summarization,openai_analysis
1007,AI image generation models,Runway ML,tried,RunwayML story in style of Wes Anderson,It’s fun to make the stories like this. \nImag...,2024-08-18 07:05:14,15,15,RunwayML,https://reddit.com/r/runwayml/comments/1ev1e3t...,NaN,NaN,Its fun to make the stories like this. Images ...,NaN
1008,AI image generation models,Runway ML,comparison,Runway ML Gen-3 Alpha prompt generator,Check this out. By far the best prompt generat...,2024-07-17 13:12:46,0,1,RunwayML,https://reddit.com/r/runwayml/comments/1e5fk4y...,NaN,NaN,Check this out. By far the best prompt generat...,NaN
1009,AI image generation models,Runway ML,opinion,Why Runway lost the game? They don't care abou...,# Why RunwayML Is Perceived as Lagging Behind ...,2025-02-26 23:06:46,18,45,RunwayML,https://reddit.com/r/runwayml/comments/1iyzq9q...,NaN,NaN,"RunwayMLs perceived lag may be temporary, but...",NaN
1010,AI image generation models,Runway ML,opinion,Dancing video with Runway ML,. Let's start this day with a bit of dance [#r...,2024-10-16 19:14:10,1,0,RunwayML,https://reddit.com/r/runwayml/comments/1g54nc1...,NaN,NaN,. Let's start this day with a bit of dance,NaN
1011,AI image generation models,Runway ML,comparison,Creating consistent Faces with Runway references,"Hi everyone, \nI’ve put together a quick guid...",2025-06-08 13:25:34,8,6,RunwayML,https://reddit.com/r/runwayml/comments/1l69weq...,NaN,NaN,"Hi everyone, Ive put together a quick guide sh...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,AI image generation models,Stable Diffusion,best settings,Best paid Stable Diffusion based online genera...,What are the best paid Stable Diffusion based ...,2024-07-22 21:53:56,1,10,aiArt,https://reddit.com/r/aiArt/comments/1e9nn5b/be...,NaN,NaN,What are the best paid Stable Diffusion based ...,NaN
1496,AI image generation models,Stable Diffusion,using,[D] Three Pilars of Intelligence,Three Pilars of Intelligence\n\nMy research jo...,2025-03-08 13:40:26,2,3,ArtificialInteligence,https://reddit.com/r/ArtificialInteligence/com...,NaN,NaN,we'll likely see evolutionary algorithms becom...,NaN
1497,AI image generation models,Stable Diffusion,tested,Is there an AI tool to change the color of obj...,"Hi guys, I was looking for a tool, preferably ...",2024-09-17 02:03:52,5,12,ArtificialInteligence,https://reddit.com/r/ArtificialInteligence/com...,NaN,NaN,"Hi guys, I was looking for a tool, preferably ...",NaN
1498,AI image generation models,Stable Diffusion,workflow,set up and run Stable Diffusion 3.5 in ComfyUI,I just wrote a guide on how to set up and run ...,2024-11-23 21:35:18,5,0,StableDiffusion,https://reddit.com/r/StableDiffusion/comments/...,NaN,NaN,I just wrote a guide on how to set up and run ...,NaN


In [16]:
df = df.dropna(subset=['openai_analysis'])

In [32]:
df['openai_analysis'].isnull().sum()

np.int64(0)

In [8]:
df.columns

Index(['category_type', 'product', 'search_term', 'title', 'text', 'date',
       'score', 'comments', 'subreddit', 'url', 'Unnamed: 10', 'Unnamed: 11',
       'summarization', 'openai_analysis'],
      dtype='object')

In [9]:
df.drop(['category_type', 'search_term', 'title', 'date', 'score', 'comments', 'subreddit', 'url', 'Unnamed: 10', 'Unnamed: 11', 'summarization'], axis=1, inplace=True)

In [10]:
df.columns

Index(['product', 'text', 'openai_analysis'], dtype='object')

In [14]:
df['openai_analysis'].head()

0    {\n  "sentiment": "Neutral",\n  "key_points": ...
1    {\n  "sentiment": "Neutral",\n  "key_points": ...
2    {\n  "sentiment": "Neutral",\n  "key_points": ...
3    {\n  "sentiment": "Neutral",\n  "key_points": ...
4    {\n  "sentiment": "Bad",\n  "key_points": ["Tw...
Name: openai_analysis, dtype: object

In [15]:
df.head()

,product,text,openai_analysis
0,DALL·E,Which tools do you think are best for a beginn...,"{\n ""sentiment"": ""Neutral"",\n ""key_points"": ..."
1,DALL·E,Hi everyone! 👋\n\nI’m looking for an AI tool t...,"{\n ""sentiment"": ""Neutral"",\n ""key_points"": ..."
2,DALL·E,* **SKYBOX AI**: create 360° worlds with one i...,"{\n ""sentiment"": ""Neutral"",\n ""key_points"": ..."
3,DALL·E,"Hi everyone!\n\nFirst off, thanks to this amaz...","{\n ""sentiment"": ""Neutral"",\n ""key_points"": ..."
4,DALL·E,When two octogenarian buddies named Nick disco...,"{\n ""sentiment"": ""Bad"",\n ""key_points"": [""Tw..."


In [17]:
df['openai_analysis'] = df['openai_analysis'].apply(json.loads)


In [38]:
df.head()

,product,text,openai_analysis
0,DALL·E,Which tools do you think are best for a beginn...,"{'sentiment': 'Neutral', 'key_points': ['The c..."
1,DALL·E,Hi everyone! 👋\n\nI’m looking for an AI tool t...,"{'sentiment': 'Neutral', 'key_points': ['The c..."
2,DALL·E,* **SKYBOX AI**: create 360° worlds with one i...,"{'sentiment': 'Neutral', 'key_points': ['Menti..."
3,DALL·E,"Hi everyone!\n\nFirst off, thanks to this amaz...","{'sentiment': 'Neutral', 'key_points': ['The u..."
4,DALL·E,When two octogenarian buddies named Nick disco...,"{'sentiment': 'Bad', 'key_points': ['Two veter..."


In [18]:
df_analisis = pd.json_normalize(df['openai_analysis'])
df = df.join(df_analisis)

In [40]:
df.head()

,product,text,openai_analysis,sentiment,key_points
0,DALL·E,Which tools do you think are best for a beginn...,"{'sentiment': 'Neutral', 'key_points': ['The c...",Neutral,[The customer is a beginner looking for the be...
1,DALL·E,Hi everyone! 👋\n\nI’m looking for an AI tool t...,"{'sentiment': 'Neutral', 'key_points': ['The c...",Neutral,[The customer is looking for an AI tool that c...
2,DALL·E,* **SKYBOX AI**: create 360° worlds with one i...,"{'sentiment': 'Neutral', 'key_points': ['Menti...",Neutral,[Mentions of various software and updates like...
3,DALL·E,"Hi everyone!\n\nFirst off, thanks to this amaz...","{'sentiment': 'Neutral', 'key_points': ['The u...",Neutral,[The user has successfully created their first...
4,DALL·E,When two octogenarian buddies named Nick disco...,"{'sentiment': 'Bad', 'key_points': ['Two veter...",Bad,"[Two veteran journalists, Nick Gage and Nick B..."


In [19]:
df.drop(['openai_analysis'], axis=1, inplace=True)

In [20]:
print(df['sentiment'].value_counts())

sentiment
Neutral    649
Good       300
Bad         58
Name: count, dtype: int64


In [43]:
print(df['product'].value_counts())

product
DALL·E         300
Leonardo AI    300
Midjourney     300
Runway ML      107
Name: count, dtype: int64


In [44]:
print(pd.crosstab(df['product'], df['sentiment']))

sentiment    Bad  Good  Neutral
product                        
DALL·E        17    94      189
Leonardo AI   18    80      202
Midjourney    19    95      186
Runway ML      4    31       72


In [21]:
df['sentiment'] = df['sentiment'].str.lower()
df = df[df['sentiment'].isin(['good', 'bad'])]

In [47]:
print(pd.crosstab(df['product'], df['sentiment']))

sentiment    bad  good
product               
DALL·E        17    94
Leonardo AI   18    80
Midjourney    19    95
Runway ML      4    31


In [22]:

print(df['sentiment'].value_counts())

sentiment
good    300
bad      58
Name: count, dtype: int64


In [54]:
print(df['key_points'].iloc[0])

['Two veteran journalists, Nick Gage and Nick Basbanes, believe that ChatGPT is stealing and repurposing their work without attribution.', 'They have filed a lawsuit against OpenAI and Microsoft, alleging systematic theft of copyrighted work.', 'The duo appreciates the technology but criticizes it for its propensity for falsehoods and lack of attribution.']


In [23]:
df['key_points'] = df['key_points'].apply(lambda kp: ' '.join(kp))

In [56]:
print(df['key_points'].iloc[0])

Two veteran journalists, Nick Gage and Nick Basbanes, believe that ChatGPT is stealing and repurposing their work without attribution. They have filed a lawsuit against OpenAI and Microsoft, alleging systematic theft of copyrighted work. The duo appreciates the technology but criticizes it for its propensity for falsehoods and lack of attribution.


In [24]:
grouped = df.groupby(['product', 'sentiment'])['key_points'].apply(lambda texts: ' '.join(texts)).reset_index()

In [25]:
grouped.head()

,product,sentiment,key_points
0,DALL·E,bad,"Two veteran journalists, Nick Gage and Nick Ba..."
1,DALL·E,good,The reviewer is seeking advice on prompts for ...
2,Leonardo AI,bad,"Images lack realism and look too clean, too sm..."
3,Leonardo AI,good,The customer is working on an open-source AI t...
4,Midjourney,bad,The customer is frustrated with the basic vers...


In [29]:
grouped

,product,sentiment,key_points
0,DALL·E,bad,"Two veteran journalists, Nick Gage and Nick Ba..."
1,DALL·E,good,The reviewer is seeking advice on prompts for ...
2,Leonardo AI,bad,"Images lack realism and look too clean, too sm..."
3,Leonardo AI,good,The customer is working on an open-source AI t...
4,Midjourney,bad,The customer is frustrated with the basic vers...
5,Midjourney,good,The reviewer has been working on a free browse...
6,Runway ML,bad,The user is struggling to generate full videos...
7,Runway ML,good,Impressed by the capabilities of the platform ...


vectorizamos con tfidf

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

# stop_words='english': elimina palabras comunes como "the", "is", etc.
# max_df=0.95: ignora palabras que aparecen en más del 95% de los textos.
# min_df=2: ignora palabras que aparecen en solo un texto (muy específicas).

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.95, min_df=2)

tfidf_matrix = vectorizer.fit_transform(grouped['key_points'])

aplicamos lda

In [27]:
from sklearn.decomposition import LatentDirichletAllocation

# numero de temas a extraer 
n_topics = 3

lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda_model.fit(tfidf_matrix)

,n_components,3
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [28]:
# Mostrar las palabras más importantes por tópico

def display_topics(model, feature_names, no_top_words=10):
    for idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
        print(f"🔹 Tópico {idx + 1}: {', '.join(top_words)}")
display_topics(lda_model, vectorizer.get_feature_names_out())

🔹 Tópico 1: dissatisfied, tools, images, midjourney, illustrator, image, lack, struggling, online, photoshop
🔹 Tópico 2: reviewer, image, video, images, dalle, tools, new, midjourney, used, open
🔹 Tópico 3: videos, finds, model, use, slow, unsatisfactory, negative, confused, stating, platforms


In [40]:
topic_keywords = display_topics(lda_model, vectorizer.get_feature_names_out())

🔹 Tópico 1: dissatisfied, tools, images, midjourney, illustrator, image, lack, struggling, online, photoshop
🔹 Tópico 2: reviewer, image, video, images, dalle, tools, new, midjourney, used, open
🔹 Tópico 3: videos, finds, model, use, slow, unsatisfactory, negative, confused, stating, platforms


In [30]:
# Obtener la distribución de tópicos para cada grupo de textos
topic_distribution = lda_model.transform(tfidf_matrix)

In [31]:
# Añadir la columna con el tópico dominante
grouped['dominant_topic'] = topic_distribution.argmax(axis=1)

In [32]:
grouped

,product,sentiment,key_points,dominant_topic
0,DALL·E,bad,"Two veteran journalists, Nick Gage and Nick Ba...",1
1,DALL·E,good,The reviewer is seeking advice on prompts for ...,1
2,Leonardo AI,bad,"Images lack realism and look too clean, too sm...",0
3,Leonardo AI,good,The customer is working on an open-source AI t...,1
4,Midjourney,bad,The customer is frustrated with the basic vers...,0
5,Midjourney,good,The reviewer has been working on a free browse...,1
6,Runway ML,bad,The user is struggling to generate full videos...,2
7,Runway ML,good,Impressed by the capabilities of the platform ...,1


In [36]:
def get_topic_keywords(lda_model, vectorizer, n_words=10):
    topic_keywords = {}
    feature_names = vectorizer.get_feature_names_out()
    for idx, topic in enumerate(lda_model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_words - 1:-1]]
        topic_keywords[idx] = top_words
    return topic_keywords

def generate_prompts(grouped_df, lda_model, vectorizer):
    topic_keywords = get_topic_keywords(lda_model, vectorizer)
    prompts = {}

    providers = grouped_df['product'].unique()

    for provider in providers:
        entry = grouped_df[grouped_df['product'] == provider]
        good_topic = entry[entry['sentiment'] == 'good']['dominant_topic'].values
        bad_topic = entry[entry['sentiment'] == 'bad']['dominant_topic'].values

        good_words = topic_keywords[good_topic[0]] if len(good_topic) else []
        bad_words = topic_keywords[bad_topic[0]] if len(bad_topic) else []

        prompt = f"""
You are a product review writer. Based on user feedback, summarize the main experiences people have with {provider}.

Positive reviews mention topics such as: {', '.join(good_words)}.

Negative reviews focus on: {', '.join(bad_words)}.

Write a short article recommending when to use {provider}, what to expect, and what potential issues to consider.
        """.strip()

        prompts[provider] = prompt

    return prompts


In [37]:
def generate_articles_from_prompts(prompts, model="gpt-4", max_tokens=400):
    results = {}

    for provider, prompt in prompts.items():
        try:
            response = openai_client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful and professional product reviewer."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=max_tokens,
                temperature=0.7
            )
            results[provider] = response.choices[0].message.content
        except Exception as e:
            results[provider] = f"Error: {e}"

    return results


In [38]:
prompts = generate_prompts(grouped, lda_model, vectorizer)

In [39]:
articles = generate_articles_from_prompts(prompts)

# Guardar en archivo plano
for provider, text in articles.items():
    with open(f"{provider.replace(' ', '_')}_article.txt", "w", encoding="utf-8") as f:
        f.write(text)


In [ ]:
print(topic_keywords)